# 実践演習9-1

畳み込みニューラルネットワークでfasion MNISTデータの識別を行います。

## 準備

必要なライブラリ等を読み込みます。

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

## データの読み込み

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

データサイズの確認

In [ ]:
train_images.shape

In [ ]:
test_images.shape

データの表示

In [ ]:
plt.imshow(train_images[0])
plt.colorbar()

ラベルとクラスとの対応

|ラベル|クラス|
|---|---|
|0|T-シャツ/トップ (T-shirt/top)|
|1|ズボン (Trouser)|
|2|プルオーバー (Pullover)|
|3|ドレス (Dress)|
|4|コート (Coat)|
|5|サンダル (Sandal)|
|6|シャツ (Shirt)|
|7|スニーカー (Sneaker)|
|8|バッグ (Bag)|
|9|アンクルブーツ (Ankle boot)|

In [ ]:
train_labels[0:20]

元データでは濃淡が0から255までで表現されており、最大値が大きすぎるので、特徴の値の最大値を1とし、型をfloatにキャストします。

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

## フィードフォワード型ネットワーク

まずベースラインとして3層のフィードフォワード型ネットワークで学習を行います。
Flattenは28×28の行列を784次元のベクトルに変換します。その後、中間層のユニット数を128、出力層のユニット数をクラス数(10)としてネットワークを構成します。

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

損失関数と最適化器を指定します。metricsは学習の進行に従って表示されるものです。

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

繰り返し回数を指定して学習を行います。

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=200)

テストデータで評価します。

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

層数を増やしてみます。

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=200)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

## 畳み込みニューラルネットワーク

データの変換（チャネル数=1を加えて4-dテンソルに）

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

ネットワークの構成の確認

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(16, kernel_size=(3, 3),
                 activation=tf.nn.relu,
                 input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation=tf.nn.relu),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.summary()

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=200)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)